## quick_pp

In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt

from quick_pp.objects import Project

# Load well from saved file
project = "MOCK_carbonate"
project_path = rf"data\04_project\{project}.qppp"
project = Project().load(project_path)
project.get_well_names()
all_data = project.get_all_data()

# Rock Typing

The process of rock typing is a multidisciplinary effort that involves close collaboration between geologists, petrophysicists, and reservoir engineers. Each discipline plays a specific role, and the rock typing workflow involves a seamless transition of data, models, and insights across these domains to build a robust understanding of the reservoir. Below is the high-level workflow detailing the steps, key tasks, and handoffs between these professionals

Petrophysicists take geological inputs and combine them with well log data and core analysis to define rock types based on petrophysical properties such as porosity, permeability, fluid saturation, and pore geometry.

Tasks:
- Core Data Integration:
    - Perform routine and special core analyses (RCAL, SCAL) to measure porosity, permeability, capillary pressure, and wettability.

- Log Interpretation:
    - Interpret wireline logs (e.g., density, neutron, resistivity, NMR) to generate continuous petrophysical properties along the wellbore.

- Rock Typing Methods:
    - Use statistical tools like Flow Zone Indicator (FZI), Pickett plots, and cluster analysis to group rocks with similar flow behavior into petrophysical rock types.

- Electrofacies Analysis:
    - Identify electrofacies by clustering log responses, especially where core data is limited.

The rock typing workflow is a collaborative process where geologists, petrophysicists, and reservoir engineers work together to ensure the reservoir is accurately characterized and modeled. This interdisciplinary approach ensures that both geological and petrophysical complexities are accounted for, resulting in more efficient production strategies and better field development plans

## FZI Rock Typing

### Identifying the number of rock types

Different methods have been discussed by previous works on determining of the number of rock types for a given core data. Among others are Wards Plot, Modified Lorenz Plot and its extensions.

1. Flow Zone Index (FZI) Method
    - The FZI method classifies rock types based on their flow characteristics. It involves calculating the Flow Zone Index from core data and using it to group similar rock types. This method is particularly effective in heterogeneous reservoirs3.

1. Ward’s Method
    - Ward’s method is a hierarchical clustering technique that minimizes the total within-cluster variance. It starts with each observation in its own cluster and merges clusters iteratively to minimize the increase in total within-cluster variance1. This method is particularly useful for quantitative variables and can be visualized using a dendrogram, which helps in identifying the optimal number of clusters (rock types).

2. Lorenz Method
    - The Lorenz method, often used in petrophysical analysis, involves plotting cumulative storage capacity against cumulative flow capacity. This method helps in identifying distinct rock types based on their flow properties. By analyzing the Lorenz plot, you can determine the number of rock types and their respective contributions to storage and flow capacities2.

However, since the lithofacies data is not available to calibrate the rock typing, Costa resolved to defining the rock types based on Winland R35 method and resulted in 27 RRTs.

For simplicity, this work break the rocks into 4 rock types only based on the Pore Throat Size (PTS) classification below;
Rock Type 1: Mega: PTS > 10 microns
Rock Type 2: Macro: PTS > 2 microns
Rock Type 3: Meso: PTS > 0.5 microns
Rock Type 4: Micro: PTS > 0.1 microns

In [ ]:
from quick_pp.rock_type import plot_ward, plot_modified_lorenz

core_data = pd.read_csv(r'data\01_raw\COSTA\HW_core_data_all.csv')
core_data['CPORE'] = core_data['Phi (frac.)']
core_data['CPERM'] = core_data['K mD']
core_data['PC'] = core_data['O/B Pc (psia)']
core_data['SW'] = core_data['Equiv Brine Sat. (Frac.)']
core_data = core_data[core_data['CPERM'] > 0]

clean_core_data = core_data.drop_duplicates(subset=['Sample', 'CPORE', 'CPERM'], keep='last')
plot_ward(clean_core_data['CPORE'], clean_core_data['CPERM'])
plot_modified_lorenz(clean_core_data['CPORE'], clean_core_data['CPERM'])

log_fzi_cut_offs = [-.679, -.179, .171, .571, .871]
fzi_cut_offs = [round(10**(i), 3) for i in log_fzi_cut_offs]
print(fzi_cut_offs)

xxx explain the core data xxx

Based on the Ward's plot above, it is deduced that the data can be grouped into 4 rock types where the limits of log(FZI) values are -0.679, -0.179, 1.71, 0.571 and 0.871.
This translates into FZI values of 0.209, 0.662, 1.483, 3.724, 7.43. Consequently, the rock types are categorized as follows;
- Rock Type 1: FZI >= 3.724
- Rock Type 2: 1.483 <= FZI < 3.724
- Rock Type 3: 0.662 <= FZI < 1.483
- Rock Type 4: FZI < 0.662

In [ ]:
from quick_pp.rock_type import plot_fzi, calc_fzi, rock_typing

# Estimate rock types
fzi = calc_fzi(all_data['CPORE'], all_data['CPERM'])
all_data['FZI'] = fzi
fzi_cut_offs = [
    .1, .2, .4, .6, .8, 1, 1.2, 1.4, 1.6, 1.8, 2, 2.2, 2.4, 2.6, 2.8, 3, 3.2, 3.4, 3.6, 3.8, 4, 4.5, 5, 6
]
fzi_rock_flag = rock_typing(fzi, fzi_cut_offs, higher_is_better=True)
all_data['ROCK_FLAG'] = fzi_rock_flag

plot_fzi(all_data['CPORE'], all_data['CPERM'], rock_type=fzi_rock_flag, cut_offs=fzi_cut_offs)
print(pd.Series(fzi_rock_flag).value_counts().sort_index())

### Develop Machine Learning models to predict ROCK_FLAG and FZI

In [ ]:
from quick_pp.rock_type import train_classification_model, train_regression_model

train_data = all_data.copy()  # Filter out low porosity data
train_data = train_data[~((train_data.CPORE <= .02) & (train_data.CPERM >= .01))].copy()  # Filter out low porosity data
# train_data = train_data[train_data.CPERM > .1]  # Filter out low permeability data
# train_data = train_data[train_data.ROCK_FLAG < 5]  # Filter out non-reservoir data

train_data['LOG_RT'] = np.log10(train_data['RT'])
train_data['NDI'] = (2.95 - train_data['RHOB']) / 1.95
# train_data['NDI_V2'] = np.log10((2.95 - train_data['RHOB']) / 1.95)

input_features = ['GR', 'NPHI', 'RHOB', 'LOG_RT', 'NDI']
train_data = train_data.dropna(subset=input_features + ['ROCK_FLAG', 'FZI'])

fzi_rt_model = train_classification_model(
    train_data, input_features=input_features, target_feature='ROCK_FLAG', stratifier=train_data['WELL_NAME'])
with open(r'data\04_project\MOCK_carbonate\outputs\fzi_rt_model.qppm', 'wb') as file:
    pickle.dump(fzi_rt_model, file)

train_data['LOG_FZI'] = np.log10(train_data['FZI'])
fzi_model = train_regression_model(
    train_data, input_features=input_features, target_feature='LOG_FZI', stratifier=train_data['WELL_NAME'])
with open(r'data\04_project\MOCK_carbonate\outputs\fzi_model.qppm', 'wb') as file:
    pickle.dump(fzi_model, file)

### Determining the perm transform parameter for each rock type.

In [ ]:
from ipywidgets import interact, widgets
from quick_pp.core_calibration import poroperm_xplot, fit_poroperm_curve

rt = widgets.Dropdown(
    options=sorted(all_data['ROCK_FLAG'].dropna().unique()),
    value=1,
    description='Rock Type:'
)

@interact(rt=rt)
def param(rt):
    data = all_data[all_data['ROCK_FLAG'] == rt]
    a, b = fit_poroperm_curve(data['CPORE'], data['CPERM'])
    poroperm_xplot(data['CPORE'], data['CPERM'], a=a, b=b, label=f'RT{rt}:\n a={a:.2f},\n b={b:.2f}')

In [ ]:
import pprint

poroperm_params = {}

for rt, data in all_data.groupby('ROCK_FLAG'):
    a, b = fit_poroperm_curve(data['CPORE'], data['CPERM'])
    poroperm_params[rt] = (a, b)

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(poroperm_params)

In [ ]:
plt.figure(figsize=(12, 8))
for rt, data in all_data.groupby('ROCK_FLAG'):
    a, b = poroperm_params[rt]
    poroperm_xplot(data['CPORE'], data['CPERM'], a=a, b=b, label=f'RT{rt}: a={a:1d}, b={b:.2f}')
    

### Compare different PERM estimations at well level

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error, r2_score

from quick_pp.core_calibration import perm_transform
from quick_pp.rock_type import calc_fzi_perm

focus_well = 'HW-26'
well_data = all_data[all_data.WELL_NAME == focus_well].copy()

# Permeability estimation based on core data perm transform
a = well_data['ROCK_FLAG'].map(poroperm_params).apply(lambda x: x[0] if type(x) == tuple else np.nan)
b = well_data['ROCK_FLAG'].map(poroperm_params).apply(lambda x: x[1] if type(x) == tuple else np.nan)
perm_trans = perm_transform(well_data['PHIT'], a=a, b=b)

# Permeability estimation based on FZI from core data
fzi = calc_fzi(well_data['CPORE'], well_data['CPERM'])
perm_fzi = calc_fzi_perm(fzi, well_data['PHIT'])

# Permeability prediction based on ROCK_FLAG ML model followed by perm transform
well_data['LOG_RT'] = np.log10(well_data['RT'])
well_data['NDI'] = (2.95 - well_data['RHOB']) / 1.95
# well_data['NDI_V2'] = np.log10((2.95 - well_data['RHOB']) / 1.95)
with open(r'data\04_project\MOCK_carbonate\outputs\fzi_rt_model.qppm', 'rb') as file:
    fzi_rt_model = pickle.load(file)
rock_flag_ml = fzi_rt_model.predict(well_data[input_features])
perm_a_ml = pd.Series(rock_flag_ml).map(poroperm_params).apply(lambda x: x[0] if type(x) == tuple else np.nan)
perm_b_ml = pd.Series(rock_flag_ml).map(poroperm_params).apply(lambda x: x[1] if type(x) == tuple else np.nan)
perm_ml = perm_transform(well_data['PHIT'], perm_a_ml, perm_b_ml)

# Permeability prediction based on FZI ML model followed by back calculate from FZI
with open(r'data\04_project\MOCK_carbonate\outputs\fzi_model.qppm', 'rb') as file:
    fzi_model = pickle.load(file)
fzi_ml = 10**(fzi_model.predict(well_data[input_features]))
perm_fzi_ml = calc_fzi_perm(fzi_ml, well_data['PHIT'])

# Plot to compare
plt.figure(figsize=(20, 2))
plt.plot(well_data.DEPTH, perm_trans, label='Perm Transform')
plt.plot(well_data.DEPTH, perm_fzi, label='Perm R35')
plt.plot(well_data.DEPTH, perm_ml, label='Perm ML')
plt.plot(well_data.DEPTH, perm_fzi_ml, label='Perm R35 ML')
plt.scatter(well_data.DEPTH, well_data.CPERM, label='Core Perm', marker='.', c='black')
plt.yscale('log')
plt.legend()

# Print scores
score_df = well_data[['DEPTH', 'CPERM']].copy()
score_df['PERM'] = perm_fzi_ml
score_df.dropna(inplace=True)
print(f"\n ### PERM MAPE: {mean_absolute_percentage_error(score_df.CPERM, score_df.PERM):.2f}")
print(f" ### PERM R2: {r2_score(score_df.CPERM, score_df.PERM):.2f}")

### Compare different ROCK_FLAG at well level

In [ ]:
# Compare rock types predicted from rt_model with applied cut-offs on predicted r35
rock_flag_fzi_ml = rock_typing(fzi_ml, fzi_cut_offs, higher_is_better=True)

plt.figure(figsize=(20, 2))
plt.plot(well_data.DEPTH, well_data.ROCK_FLAG, label='Rock Flag Core')
plt.plot(well_data.DEPTH, rock_flag_ml, label='Rock Flag ML')
plt.plot(well_data.DEPTH, rock_flag_fzi_ml, label='Rock Flag FZI ML')
plt.legend()


## Plotting the Rock Types

As a comparison, the rock types being identified using FZI is plotted on Winland R35 and Lucia Rock Fabric Number (RFN) methods.

- The plot demonstrates Winland R35 resulted in a more flat permeability estimation while
- Lower porosity - high permeability datapoints.
    - These points might be indicating fracture kind of rock types
    - Both Winland R35 and Lucia RFN does not model the datapoints
    - FZI is able to model but indicates too high of a value

In [ ]:
from quick_pp.rock_type import plot_winland, plot_rfn

plot_fzi(all_data['CPORE'], all_data['CPERM'], rock_type=fzi_rock_flag)
plot_winland(all_data['CPORE'], all_data['CPERM'], rock_type=fzi_rock_flag)
plot_rfn(all_data['CPORE'], all_data['CPERM'], rock_type=fzi_rock_flag)

# Plotting the result

In [ ]:
from quick_pp.plotter import plotly_log

# Plot individual results
well_data['PERM'] = perm_fzi_ml
fig = plotly_log(well_data, 'ft')
fig.show(config=dict(scrollZoom=True))

In [ ]:
# # Save the well data
# project.update_data(well_data)
# project.save()